In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops'

Model training 

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [7]:
from src.mlproject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.mlproject.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filapath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filapath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name= config.model_name,
            l1_ratio= params.l1_ratio,
            alpha=params.alpha,
            target_column=schema.name)
        
        return model_trainer_config

In [9]:
import pandas as pd
import os
from src.mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self,
                 config : ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop(self.config.target_column, axis=1)
        y_train = train_data[[self.config.target_column]]
        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio,
                        random_state=42)
        lr.fit(X_train,y_train)
        
        ## model has been prepared
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        model_path = model_path.replace( "/",'\\')
        print("Model Path:", model_path)
        joblib.dump(lr, model_path)

        
    
  

In [11]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

2023-09-17 20:44:03,390 - common - INFO - yaml file : {} loaded successfully
2023-09-17 20:44:03,398 - common - INFO - yaml file : {} loaded successfully
2023-09-17 20:44:03,404 - common - INFO - yaml file : {} loaded successfully
2023-09-17 20:44:03,408 - common - INFO - Created directory : artifacts
2023-09-17 20:44:03,410 - common - INFO - Created directory : artifacts/model_trainer


Model Path: artifacts\model_trainer\model.joblib
